In [ ]:
## Initial imports
import pandas as pd
from pathlib import Path
import hvplot.pandas
import os
import seaborn as sns
import pygwalker as pyg
import numpy as np

## Data Import for Anaysis

Read csv files from different sources into  Dataframes and Clean Data

In [ ]:
#Read the US Dollar Index Historical Data, Group into yearly data, rename columns and clean Data.
us_dollar_df = pd.read_csv('./Resources/US_Dollar_Index_Historical_Data_1971-2023.csv')
us_dollar_df['Date'] = pd.DatetimeIndex(us_dollar_df['Date']).year
us_dollar_df = us_dollar_df.rename(columns = {"Date":"Year","Price":"Dollar Price"})
us_dollar_df.head()

In [ ]:
us_dollar_df = us_dollar_df.groupby("Year").mean().round(2)
us_dollar_df.head()

In [ ]:
us_dollar_df = us_dollar_df[['Dollar Price']]
us_dollar_df.head()

In [ ]:
#Read the US Globalization Data, Set Year as index and clean Data.
us_globalization_df = pd.read_csv('./Resources/US_Globalization_1971-2021.csv')
us_globalization_df = us_globalization_df.set_index('Year').round(2)
us_globalization_df.head()

In [ ]:
#Read the US National Debt Data, Set Year as index and clean Data.
us_national_df = pd.read_csv('./Resources/US_National_Debt_by_the_Year_1971-2023.csv', encoding='unicode_escape')
us_national_df = us_national_df.set_index('Year')
us_national_df['DEBT'] = us_national_df['DEBT'].str.replace("$", "")
us_national_df['DEBT'] = us_national_df['DEBT'].str.replace(",", "")
us_national_df['DEBT'] = us_national_df['DEBT'].astype("float")
us_national_df = us_national_df[['DEBT']]
us_national_df.head()

In [ ]:
#Read the US Oil Prices 1971 Data, Set Year as index and clean Data.
us_oil_71_df = pd.read_csv('./Resources/US_Oil_Prices_1971-2022.csv', encoding='unicode_escape')
us_oil_71_df = us_oil_71_df.set_index("Year").rename(columns = {"Average":"Avg Oil Prices"})
us_oil_71_df['Avg Oil Prices'] = us_oil_71_df['Avg Oil Prices'].str.replace("$", "")
us_oil_71_df['Avg Oil Prices'] = us_oil_71_df['Avg Oil Prices'].astype("float")
us_oil_71_df = us_oil_71_df[['Avg Oil Prices']]
us_oil_71_df.head()

In [ ]:
#Read the US ResearchandDev Data, Rename column, Set Year as index and clean Data.
us_r_and_d_df = pd.read_csv('./Resources/US_ResearchandDev_1971-2023.csv')
us_r_and_d_df['Percentage of Fiscal Budget for R&D'] = us_r_and_d_df['Percentage of Fiscal Budget for R&D'].str.replace("%","")
us_r_and_d_df['Percentage of Fiscal Budget for R&D'] = pd.to_numeric(us_r_and_d_df['Percentage of Fiscal Budget for R&D'])
us_r_and_d_df= us_r_and_d_df.rename(columns = {"Fiscal Year":"Year"}).set_index("Year")
us_r_and_d_df= us_r_and_d_df[['Percentage of Fiscal Budget for R&D']]
us_r_and_d_df.head()

In [ ]:
data_df =  pd.concat([us_dollar_df, us_national_df, us_oil_71_df, us_globalization_df, us_r_and_d_df], axis='columns', join='inner')
data_df.head()

## Performance Analysis - 1st Iteration

Conduct performance analysis to deduce the correlation between a single Variable (Oil Price) and its impact to the US dollar index

In [ ]:
# Join the US Dollar index and the Oil Prices dataframe
combined_data_df = pd.concat(
    [us_dollar_df, us_oil_71_df], axis='columns', join='inner'
)
combined_data_df.head()

In [ ]:
combined_data_df.hvplot(title= "Dollar Price Index Vs. Average Oil Price")

In [ ]:
# Calcuate the percentatge difference YOY for the US Dollar Index and Avg Oil Prices
diff_yoy_combined_data_df = combined_data_df.pct_change().dropna()
diff_yoy_combined_data_df.head()

In [ ]:
diff_yoy_combined_data_df.plot(kind="bar", title= "Diff YoY between Oil Price and Dollar Index")

In [ ]:
# Calculate the correlation between US Index and Oil Price
corr_combined_data_df = diff_yoy_combined_data_df.corr()
corr_combined_data_df

In [ ]:
# Display the correlation matrix
sns.heatmap(corr_combined_data_df, vmin =-1, vmax = 1)

In [ ]:
# percentage change of all factors
pct_change_data_df = data_df.pct_change()

In [ ]:
correlation = pct_change_data_df.corr()
sns.heatmap(correlation, vmin =-1, vmax = 1, annot = True, cmap="BuPu")

## Correlation Across Different Variables

In [ ]:
correlation

In [ ]:
pyg.walk(data_df)

In [ ]:
# Dollar Price	1.000000	-0.113371	-0.095077	-0.328763	-0.166765
dollar_vs_factor_correlation_df = pd.DataFrame({'factors' : ['DEBT', 'Avg Oil Prices','Trade (% of GDP)','Percentage of Fiscal Budget for R&D'], 'correlation' : [-0.113371,-0.095077, -0.328763,-0.166765]})
dollar_vs_factor_correlation_df

In [ ]:
pyg.walk(diff_yoy_combined_data_df)



In [ ]:
## Yearly Monte Carlo to Reduce Noise in Monte Carlo Simulation

In [ ]:
# % change in yearly USD
yearly_pct_change_usd = us_dollar_df['Dollar Price'].pct_change().fillna(0)
yearly_pct_change_usd

In [ ]:
# The Monte Carlo simulation depicts a probability distribution of random outcomes of the USD valuation across the 1971-2023 period on top of the actual distribution of the % change of the USD
# x-axis = number of years during the period 1971-2023
# y-axis = %change movement of the USD
montecarlo2 = yearly_pct_change_usd.montecarlo(sims=30, bust=-1, goal=1)
montecarlo2.plot(title="USD Monte Carlo Simulation based on Yearly Data")

In [ ]:
# summary statistics
print(montecarlo2.stats)

## Monthly Monte Carlo Simulation

In [ ]:
#Using the monthly data from the US dollar index data frame to get a more precise analysis
us_dollar_df = pd.read_csv('./Resources/US_Dollar_Index_Historical_Data_1971-2023.csv')
us_dollar_df['Date'] = pd.DatetimeIndex(us_dollar_df['Date'])
us_dollar_df = us_dollar_df.rename(columns = {"Price":"Dollar Price"})
us_dollar_df.head()

In [ ]:
monhtly_us = pd.read_csv('./Resources/US_Dollar_Index_Historical_Data_1971-2023.csv')
monhtly_us['Date'] = pd.DatetimeIndex(us_dollar_df['Date'])
monhtly_us

In [ ]:
# monthly % change USD
pct_change_usd = monhtly_us['Price'].pct_change().fillna(0)
pct_change_usd



In [ ]:
import pandas_montecarlo

In [ ]:
# x-axis = number of months during the period 1971-2023
#y-axis = %change movement of the USD
montecarlo = pct_change_usd.montecarlo(sims=30, bust=-1, goal=1)
montecarlo.plot(title="USD Monte Carlo Simulation based on Monthly Data")

In [ ]:
# summary statistics
print(montecarlo.stats)



In [ ]:
print(montecarlo.maxdd)



Analysis
* 2 trends are depicted on the Monte Carlo:
1. Long-Term Trend: Increasing Value of the USD over the 50y period.
Looking at the Monthly Monte Carlo, overall, over the last 50 years, the USD has been increasing in value
2. Short-Term Cyclical Trend: 15 years cycles of expansion, peak, and contraction.
This trend is apparent in the yearly Monte Carlo (1st cycle: y10-y25, 2nd cycle: y25-y40, 3rd cycle: y40-..?)
Conclusion/Prediction: Currently, we are about 10 years into the USD cycle and reaching a peak. As a result, based on the observed pattern, we can expect the USD to enter into a contraction phase.¶Meaning we expect the USD to decrease in value, which confirms our hypothesis.

## Probability Distribution of the USD

In [ ]:
pct_change_usd.hist()

 ## Portfolio Analysis

In [ ]:
# import GLD csv
gold_df = pd.read_csv('./Resources/GLD.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)
gold_df

In [ ]:
#Clean Gold Data
gold_df = gold_df.drop('Open', axis='columns')
gold_df = gold_df.drop('High', axis='columns')
gold_df = gold_df.drop('Low', axis='columns')
gold_df = gold_df.drop('Adj Close', axis='columns')
gold_df = gold_df.drop('Volume', axis='columns')
gold_df.rename(columns = {"Close" : "Gold Close"}, inplace = True)
gold_df

In [ ]:
# Calculate Gold Percent Change and Drop NAs
gold_df_change = gold_df.pct_change()
gold_df_change = gold_df_change.dropna()
gold_df_change.rename(columns = {"Gold Close" : "Gold Percent Change"}, inplace = True)
gold_df_change

In [ ]:
# import CVGI csv
cvgi_df = pd.read_csv('./Resources/CVGI.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)
cvgi_df

In [ ]:
#Clean CVIG Data
cvgi_df = cvgi_df.drop('Open', axis='columns')
cvgi_df = cvgi_df.drop('High', axis='columns')
cvgi_df = cvgi_df.drop('Low', axis='columns')
cvgi_df = cvgi_df.drop('Adj Close', axis='columns')
cvgi_df = cvgi_df.drop('Volume', axis='columns')
cvgi_df.rename(columns = {"Close" : "CVGI Close"}, inplace = True)
cvgi_df

In [ ]:
# Calculate CVIG Percent Change and Drop NAs
CVIG_df_change = cvgi_df.pct_change()
CVIG_df_change = cvgi_df.dropna()
CVIG_df_change

In [ ]:
CVIG_df_change.rename(columns = {"CVIG Close" : "CVIG Percent Change"}, inplace = True)
CVIG_df_change

In [ ]:
# import CVRX csv
cvrx_df = pd.read_csv('./Resources/CVRX.csv',index_col = "Date", parse_dates = True, infer_datetime_format = True)
cvrx_df

In [ ]:
#Clean CVRX Data
cvrx_df = cvrx_df.drop('Open', axis='columns')
cvrx_df = cvrx_df.drop('High', axis='columns')
cvrx_df = cvrx_df.drop('Low', axis='columns')
cvrx_df = cvrx_df.drop('Adj Close', axis='columns')
cvrx_df = cvrx_df.drop('Volume', axis='columns')
cvrx_df.rename(columns = {"Close" : "CVRX Close"}, inplace = True)
cvrx_df

In [ ]:
# Calculate CVXR Percent Change and Drop NAs
CVXR_df_change = cvrx_df.pct_change()
CVXR_df_change = cvrx_df.dropna()
CVXR_df_change.rename(columns = {"CVXR Close" : "CVXR Percent Change"}, inplace = True)
CVXR_df_change

In [ ]:
# import LIDR csv
lidr_df = pd.read_csv('./Resources/LIDR.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)
lidr_df

In [ ]:
#Clean LIDR Data
lidr_df = lidr_df.drop('Open', axis='columns')
lidr_df = lidr_df.drop('High', axis='columns')
lidr_df = lidr_df.drop('Low', axis='columns')
lidr_df = lidr_df.drop('Adj Close', axis='columns')
lidr_df = lidr_df.drop('Volume', axis='columns')
lidr_df.rename(columns = {"Close" : "LIDR Close"}, inplace = True)
lidr_df

In [ ]:
# Calculate LIDR Percent Change and Drop NAs
lidr_df_change = lidr_df.pct_change()
lidr_df_change = lidr_df.dropna()
lidr_df_change.rename(columns = {"LIDR Close" : "LIDR Percent Change"}, inplace = True)
lidr_df_change

In [ ]:
# import APEN csv
apen_df = pd.read_csv('./Resources/APENX.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)
apen_df

In [ ]:
#Clean APEN Data
apen_df = apen_df.drop('Open', axis='columns')
apen_df = apen_df.drop('High', axis='columns')
apen_df = apen_df.drop('Low', axis='columns')
apen_df = apen_df.drop('Adj Close', axis='columns')
apen_df = apen_df.drop('Volume', axis='columns')
apen_df.rename(columns = {"Close" : "APEN Close"}, inplace = True)
apen_df

In [ ]:
# Calculate APEN Percent Change and Drop NAs
apen_df_change = apen_df.pct_change()
apen_df_change = apen_df.dropna()
apen_df_change.rename(columns = {"APEN Close" : "APEN Percent Change"}, inplace = True)
apen_df_change

In [ ]:
# import STIM csv 
stim_df = pd.read_csv('./Resources/NFT9816-USD.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)
stim_df

In [ ]:
#Clean STIM Data
stim_df = stim_df.drop('Open', axis='columns')
stim_df = stim_df.drop('High', axis='columns')
stim_df = stim_df.drop('Low', axis='columns')
stim_df = stim_df.drop('Adj Close', axis='columns')
stim_df = stim_df.drop('Volume', axis='columns')
stim_df.rename(columns = {"Close" : "STIM Close"}, inplace = True)
stim_df

In [ ]:
# Calculate STIM Percent Change and Drop NAs
stim_df_change = stim_df.pct_change()
stim_df_change = stim_df.dropna()
stim_df_change.rename(columns = {"STIM Close" : "STIM Percent Change"}, inplace = True)
stim_df_change

In [ ]:
# import TM csv
tm_df = pd.read_csv('./Resources/TM.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)
tm_df

In [ ]:
#Clean TM Data
tm_df = tm_df.drop('Open', axis='columns')
tm_df = tm_df.drop('High', axis='columns')
tm_df = tm_df.drop('Low', axis='columns')
tm_df = tm_df.drop('Adj Close', axis='columns')
tm_df = tm_df.drop('Volume', axis='columns')
tm_df.rename(columns = {"Close" : "TM Close"}, inplace = True)
tm_df

In [ ]:
# Calculate TM Percent Change and Drop NAs
tm_df_change = tm_df.pct_change()
tm_df_change = tm_df.dropna()
tm_df_change.rename(columns = {"TM Close" : "TM Percent Change"}, inplace = True)
tm_df_change

In [ ]:
# import ZYXI csv
zyxi_df = pd.read_csv('./Resources/ZYXI.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)
zyxi_df

In [ ]:
#Clean ZYXI Data
zyxi_df = zyxi_df.drop('Open', axis='columns')
zyxi_df = zyxi_df.drop('High', axis='columns')
zyxi_df = zyxi_df.drop('Low', axis='columns')
zyxi_df = zyxi_df.drop('Adj Close', axis='columns')
zyxi_df = zyxi_df.drop('Volume', axis='columns')
zyxi_df.rename(columns = {"Close" : "ZYXI Close"}, inplace = True)
zyxi_df

In [ ]:
# Calculate ZYXI Percent Change and Drop NAs
zyxi_df_change = zyxi_csv.pct_change()
zyxi_df_change = zyxi_csv.dropna()
zyxi_df_change.rename(columns = {"ZYXI Close" : "ZYXI Percent Change"}, inplace = True)
zyxi_df_change

## Daily Close Combined

In [ ]:
daily_return = pd.concat([CVIG_df_change, CVXR_df_change, gold_df_change, lidr_df_change, apen_df_change, stim_df_change, tm_df_change, zyxi_df_change], axis = 1, join = "inner")
daily_return

In [ ]:
daily_return.plot(figsize=(15,5), title ='Daily Return of Stocks')

In [ ]:
daily_return = daily_close.pct_change()
daily_return = daily_return.dropna()
daily_return

In [ ]:
# import S&P 500
sp500_df = pd.read_csv('./Resources/^SPX.csv', index_col = "Date", parse_dates = True, infer_datetime_format = True)

## Portfolio Optimization: Max Sharpe-Ratio

In [ ]:
## Calculate the weighted returns for the portfolio, to max the Sharpe ratio we're only using Zynex, Apollo Endosurgery, and Gold
## This investment decision was determined thanks to a portfolio optimization tool (see ReadMe)

In [ ]:
# Set weights: to optimize the portfolio as to maximize the sharpe ratio we only invest in  Zynex, Apollo Endosurgery, and Gold
weights = [0, 0, 0.5489, 0, 0.0219, 0, 0, 0.4291]
# Calculate portfolio return
portfolio_return = daily_return.dot(weights)
# Display sample data
portfolio_return

In [ ]:
# Calculate Annualized Sharpe Ratios

In [ ]:
# sharpe ratios calculated without risk-free data
sharpe_ratio = (portfolio_return.mean() * 252) / (portfolio_return.std() * np.sqrt(252))
sharpe_ratio

In [ ]:
# A Sharpe ratio of 1.02 indicates that the investment is generating 1.02 units of excess return for each unit of risk taken, relative to the risk-free rate.

# Calculate cumulative returns of the portfolio over the past 7 months
cummulative_portfolio_return = (1 + portfolio_return).cumprod()-1
cummulative_portfolio_return